In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import joblib
import json

In [18]:
#import os
#print(os.getcwd())

In [19]:
#encoded + cleaned data
df = pd.read_csv("../data/processed/marketing_campaign_cleaned.csv")
df_encoded = pd.get_dummies(df, drop_first=True)

X = df_encoded.drop("Response", axis=1)
y = df_encoded["Response"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [20]:
#SMOTE
sm = SMOTE(random_state=42)
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)

#XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_sm, y_train_sm)

#evaluate
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:, 1]

/Volumes/Projects/Portfolio/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [21:55:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [21]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

[[366  15]
 [ 33  34]]
              precision    recall  f1-score   support

           0       0.92      0.96      0.94       381
           1       0.69      0.51      0.59        67

    accuracy                           0.89       448
   macro avg       0.81      0.73      0.76       448
weighted avg       0.88      0.89      0.89       448

AUC: 0.8959533043444197


In [24]:
#XGB model save
joblib.dump(xgb, "../models/xgboost_response_model.pkl")

with open("../models/feature_columns.json", "w") as f:
    json.dump(list(X.columns), f)
